In [1]:
using Pkg 
using XLSX
using DataFrames
using XLSX
using Statistics
using Clustering
using Distances
using JuMP
using PlotlyJS


In [2]:
path = "/cluster/home/danare/git/Clustering/data/Hourly_Data_Europe_v13_kl_13_12_2023_windDataWP1.xlsx"

Country_Data_Entries_full = 
[
    "LOAD", "PV_AVG", "PV_INF", "PV_OPT", "PV_TRA",
    "WIND_ONSHORE_AVG", "WIND_ONSHORE_INF", "WIND_ONSHORE_OPT",
    "WIND_OFFSHORE", "WIND_OFFSHORE_SHALLOW", "WIND_OFFSHORE_DEEP",
    "MOBILITY_PSNG", "HEAT_LOW", "HEAT_HIGH", "HP_AIRSOURCE", "HP_GROUNDSOURCE",
    "HYDRO_ROR"
]

Country_Data_Entries = 
[
    "LOAD", "PV_AVG", "WIND_ONSHORE_AVG", "WIND_OFFSHORE_SHALLOW"
]

countries = [:AT, :BE, :CH, :CZ, :DE, :EE, :FI, :FR, :ES, :IE, :LT, :LU, :LV, :NL, :PL, :UK, :NO1, :NO2, :NO3, :NO4, :NO5, :DK1, :DK2, :SE1, :SE2, :SE3, :SE4];

In [3]:
##### LOAD THE DATA

CountryData = Dict()
hourly_data = XLSX.readxlsx(path)


for cde ∈ Country_Data_Entries
    CountryData[cde] = DataFrame(XLSX.gettable(hourly_data[cde]))
    select!(CountryData[cde], (countries))
end


# reformat the load
for r ∈ countries
    CountryData["LOAD"][:,r]  = CountryData["LOAD"][:,r]  / 8760

end 

In [4]:
# NORMALIZE THE DATA Z-NOMRALIZATION
function zscore_column!(col)
    mean_col = mean(col)
    std_col = std(col)
    return (col .- mean_col) / std_col
end

for cde ∈ Country_Data_Entries
    for col in names(CountryData[cde])
        CountryData[cde][!, col] .= zscore_column!(CountryData[cde][!, col])
    end
end

In [5]:
# create the matrix for the kmeans algorithms
i = 1
x = 0
for cde in Country_Data_Entries
    for n in names(CountryData[cde])
        if i == 1
            x = reshape(CountryData[cde][:,n], (365,24))'
        else
            x = vcat(x, reshape(CountryData[cde][:,n], (365,24))')
        end
        i += 1
    end
end

### Apply K-Means

In [6]:
x = convert(Matrix{Float64}, x)
x[isnan.(x)] .= 0.0  # Replace NaN values with 0.0, or handle them appropriately

43800-element view(::Vector{Float64}, [1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954  …  945831, 945832, 945833, 945834, 945835, 945836, 945837, 945838, 945839, 945840]) with eltype Float64:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [7]:
k = 6
R = kmeans(x, k)
a = assignments(R) # get the assignments of points to clusters
c = counts(R) # get the cluster sizes
M = R.centers
println(typeof(R), typeof(a), typeof(c), typeof(M))

KmeansResult{Matrix{

Float64}, Float64, Int64}Vector{Int64}Vector{Int64}Matrix{Float64}


In [8]:
FullData = JuMP.Containers.DenseAxisArray(zeros(length(countries), length(Country_Data_Entries), 6,24), countries, Country_Data_Entries, 1:6, 1:24) 

for d in 1:6
    for (m,t) in enumerate(Country_Data_Entries)
        for (b,c) in enumerate(countries)
            a = max((m-1)*length(countries)*24+(((b-1)*24)+1),1)
            println(a)
            FullData[c,t,d,:] = M[a:a+23,d]
        end
    end
end

1


25
49
73
97
121
145
169
193
217
241
265
289
313
337
361
385
409
433
457
481
505
529
553
577
601
625
649
673
697
721
745
769
793
817
841
865
889
913
937
961
985
1009
1033
1057
1081
1105
1129
1153
1177
1201
1225
1249
1273
1297
1321
1345
1369
1393
1417
1441
1465
1489
1513
1537
1561
1585
1609
1633
1657
1681
1705
1729
1753
1777
1801
1825
1849
1873
1897
1921
1945
1969
1993
2017
2041
2065
2089
2113
2137
2161
2185
2209
2233
2257
2281
2305
2329
2353
2377
2401
2425
2449
2473
2497
2521
2545
2569
1
25
49
73
97
121
145
169
193
217
241
265
289
313
337
361
385
409
433
457
481
505
529
553
577
601
625
649
673
697
721
745
769
793
817
841
865
889
913
937
961
985
1009
1033
1057
1081
1105
1129
1153
1177
1201
1225
1249
1273
1297
1321
1345
1369
1393
1417
1441
1465
1489
1513
1537
1561
1585
1609
1633
1657
1681
1705
1729
1753
1777
1801
1825
1849
1873
1897
1921
1945
1969
1993
2017
2041
2065
2089
2113
2137
2161
2185
2209
2233
2257
2281
2305
2329
2353
2377
2401
2425
2449
2473
2497
2521
2545
2569
1
25
49
73
97
121


In [9]:
## plot the results
country = "DE"
Country_Data_Entries

p = make_subplots(rows=length(Country_Data_Entries), cols=6, vertical_spacing=0.02, column_titles=["Day$i" for i in 1:6], row_titles=Country_Data_Entries)

# real da
for (i,t) in enumerate(Country_Data_Entries)
    for (k,m) in enumerate(0:24:8735)
        add_trace!(p, 
        scatter(y=CountryData[t][m+1:m+24,:DE], 
        marker=attr(
            color="grey"
        ),
        showlegend=false,
        ), 
        row=i, 
        col=a[k+1])
    end
end


# clustered data
for d in 1:6
    for (i,t) in enumerate(Country_Data_Entries)
        add_trace!(p, 
        scatter(y=FullData[:DE,t,d,:], 
        marker=attr(
            color="red"
        ),
        showlegend=false,
        ), 
        row=i, 
        col=d)
    end
end

      

relayout!(p)
p


BoundsError: BoundsError: attempt to access Int64 at index [2]

In [11]:
for (k,t) in enumerate(1:length(countries)*24:length(countries)*length(Country_Data_Entries)*24)
    print(k,t)
end

1126493129741945

In [12]:
a = assignments(R)

365-element Vector{Int64}:
 6
 6
 6
 3
 3
 3
 3
 3
 5
 5
 ⋮
 4
 4
 4
 6
 6
 6
 6
 3
 3

In [13]:
R

KmeansResult{Matrix{Float64}, Float64, Int64}([0.4924434242837145 0.9866986830975873 … 0.9758253308860382 -0.6650812505099463; -0.2972232232554791 0.9461661445063825 … 1.3561638915152878 0.35378088504977945; … ; 0.03788914574968966 0.22090747113572815 … 0.4478099347545115 0.17249327424486266; 0.3410738767269774 -0.2805557596704696 … 0.0031259725887261935 0.30492321066222217], [6, 6, 6, 3, 3, 3, 3, 3, 5, 5  …  4, 4, 4, 4, 6, 6, 6, 6, 3, 3], [1480.42591131374, 1455.7743172102255, 1567.5922261874193, 1653.8740863544526, 1447.2414529044308, 1347.3739997249972, 1398.8097698444371, 1563.9493814954722, 1600.7599406719892, 1384.9078841482215  …  1259.8337705082386, 1216.0539581817784, 1319.033865060373, 1496.8398165449876, 1520.204011459913, 1357.7766629861408, 1353.5494110649852, 1502.7002209387551, 1686.4465669743922, 1528.2708837047321], [21, 71, 77, 59, 72, 65], [21, 71, 77, 59, 72, 65], 450619.72873282194, 8, true)

In [14]:
a[7]

3

In [15]:
for i in 0:24:8760
    println(i)
end

0
24
48
72
96
120
144
168
192
216
240
264
288
312
336
360
384
408
432
456
480
504
528
552
576
600
624
648
672
696
720
744
768
792
816
840
864
888
912
936
960
984
1008
1032
1056
1080
1104
1128
1152
1176
1200
1224
1248
1272
1296
1320
1344
1368
1392
1416
1440
1464
1488
1512
1536
1560
1584
1608
1632
1656
1680
1704
1728
1752
1776
1800
1824
1848
1872
1896
1920
1944
1968
1992
2016
2040
2064
2088
2112
2136
2160
2184
2208
2232
2256
2280
2304
2328
2352
2376
2400
2424
2448
2472
2496
2520
2544
2568
2592
2616
2640
2664
2688
2712
2736
2760
2784
2808
2832
2856
2880
2904
2928
2952
2976
3000
3024
3048
3072
3096
3120
3144
3168
3192
3216
3240
3264
3288
3312
3336
3360
3384
3408
3432
3456
3480
3504
3528
3552
3576
3600
3624
3648
3672
3696
3720
3744
3768
3792
3816
3840
3864
3888
3912
3936
3960
3984
4008
4032
4056
4080
4104
4128
4152
4176
4200
4224
4248
4272
4296
4320
4344
4368
4392
4416
4440
4464
4488
4512
4536
4560
4584
4608
4632
4656
4680
4704
4728
4752
4776
4800
4824
4848
4872
4896
4920
4944
4968
4992
501

In [16]:
Country_Data_Entries

4-element Vector{String}:
 "LOAD"
 "PV_AVG"
 "WIND_ONSHORE_AVG"
 "WIND_OFFSHORE_SHALLOW"

In [17]:
CountryData["LOAD"]

Row,AT,BE,CH,CZ,DE,EE,FI,FR,ES,IE,LT,LU,LV,NL,PL,UK,NO1,NO2,NO3,NO4,NO5,DK1,DK2,SE1,SE2,SE3,SE4
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-0.769321,-0.813973,0.708818,-1.16224,-1.70109,-0.18246,0.2894,0.129599,-0.878207,-0.0239478,-0.652896,-2.04885,-0.850718,-0.828672,-1.05033,-0.612045,0.0908224,0.0908224,0.0908224,0.0908224,0.0908224,-0.886709,-0.666572,0.329907,-0.115752,-0.272717,-0.428983
2,-1.01049,-1.1672,0.581565,-1.24871,-1.84247,-0.299649,0.0982983,0.0809603,-1.08189,-0.239817,-0.861816,-2.13868,-1.02632,-0.982951,-1.29608,-0.529826,0.0339132,0.0339132,0.0339132,0.0339132,0.0339132,-0.922147,-0.753228,0.328338,-0.155141,-0.308289,-0.46991
3,-1.18211,-1.49248,0.393891,-1.28394,-1.98977,-0.455902,-0.034533,-0.138565,-1.41618,-0.611312,-1.05368,-2.26935,-1.18064,-1.20702,-1.53423,-0.609538,-0.0220928,-0.0220928,-0.0220928,-0.0220928,-0.0220928,-0.913923,-0.774417,0.318662,-0.192434,-0.335339,-0.505179
4,-1.39468,-1.75124,0.0807951,-1.38243,-2.1297,-0.573092,-0.102202,-0.412857,-1.72527,-0.986154,-1.16454,-2.35919,-1.26578,-1.42415,-1.70806,-0.87797,-0.0720766,-0.0720766,-0.0720766,-0.0720766,-0.0720766,-0.864573,-0.770268,0.340226,-0.183122,-0.32684,-0.530369
5,-1.40833,-1.87697,-0.257934,-1.44488,-2.1815,-0.582857,-0.0527033,-0.547283,-1.91951,-1.21708,-1.20291,-2.42452,-1.20724,-1.55925,-1.76263,-1.13413,-0.0744854,-0.0744854,-0.0744854,-0.0744854,-0.0744854,-0.741837,-0.740645,0.462389,-0.121994,-0.231857,-0.495835
6,-1.36087,-1.88096,-0.370539,-1.53536,-2.2387,-0.548677,0.0619576,-0.578458,-1.99069,-1.38777,-1.14322,-2.41635,-1.19128,-1.58496,-1.81105,-1.35254,-0.0687644,-0.0687644,-0.0687644,-0.0687644,-0.0687644,-0.488068,-0.487859,0.650923,0.0361698,0.0202875,-0.240046
7,-1.33162,-1.85102,-0.507862,-1.59461,-2.4052,-0.494965,0.055692,-0.570415,-1.98985,-1.47981,-1.08779,-2.49802,-1.13274,-1.51639,-1.79768,-1.42632,0.0333109,0.0333109,0.0333109,0.0333109,0.0333109,-0.185745,-0.0977326,0.796581,0.23598,0.290832,0.122406
8,-1.17171,-1.99404,-0.701029,-1.63785,-2.41319,-0.407073,0.157822,-0.567122,-1.94974,-1.45136,-1.09205,-2.39185,-1.12742,-1.38987,-1.8808,-1.35914,0.188983,0.188983,0.188983,0.188983,0.188983,0.0121107,0.192047,0.825099,0.332097,0.420733,0.315216
9,-0.968889,-1.94349,-0.763282,-1.63144,-2.33326,-0.382659,0.140278,-0.603045,-2.00455,-1.44634,-0.968407,-2.32652,-0.919894,-1.24294,-1.9238,-1.33143,0.295274,0.295274,0.295274,0.295274,0.295274,0.0651052,0.392436,0.824458,0.415484,0.516892,0.432873


In [18]:
8760-24

8736

In [19]:
for i in 0:24:8759
    println(i)
end

0
24
48
72
96
120
144
168
192
216
240
264
288
312
336
360
384
408
432
456
480
504
528
552
576
600
624
648
672
696
720
744
768
792
816
840
864
888
912
936
960
984
1008
1032
1056
1080
1104
1128
1152
1176
1200
1224
1248
1272
1296
1320
1344
1368
1392
1416
1440
1464
1488
1512
1536
1560
1584
1608
1632
1656
1680
1704
1728
1752
1776
1800
1824
1848
1872
1896
1920
1944
1968
1992
2016
2040
2064
2088
2112
2136
2160
2184
2208
2232
2256
2280
2304
2328
2352
2376
2400
2424
2448
2472
2496
2520
2544
2568
2592
2616
2640
2664
2688
2712
2736
2760
2784
2808
2832
2856
2880
2904
2928
2952
2976
3000
3024
3048
3072
3096
3120
3144
3168
3192
3216
3240
3264
3288
3312
3336
3360
3384
3408
3432
3456
3480
3504
3528
3552
3576
3600
3624
3648
3672
3696
3720
3744
3768
3792
3816
3840
3864
3888
3912
3936
3960
3984
4008
4032
4056
4080
4104
4128
4152
4176
4200
4224
4248
4272
4296
4320
4344
4368
4392
4416
4440
4464
4488
4512
4536
4560
4584
4608
4632
4656
4680
4704
4728
4752
4776
4800
4824
4848
4872
4896
4920
4944
4968
4992
501